# 2_네이버뉴스
# (4)감정분석

직접 크롤링 => 라벨링되어 있지 않은 상태  
  
1. 감성사전 기반  
- 각 기사 제목에서 단어를 추출한 뒤 단어별 점수를 합산
- 점수 합계 기준으로 -1(부정), 0(중립), 1(긍정) 분류  
2. 사전학습 모델 사용  
- 이미 방대한 데이터로 훈련되어있음 (+파인튜닝)
- ex) KoBERT  
3. chat gpt api로 영어 번역시킨 뒤 감정분석 라이브러리 사용  
- ex) TextBlob, vader


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
naver_news = pd.read_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/preprocessed_naver_news.csv")

In [ ]:
naver_news = naver_news[['month', 'title', 'token_lst', 'token']]

In [ ]:
naver_news

,month,title,token_lst,token
0,2023-01,볼보 폴스타 제쳤다 안전한 자동차 위 오른 국산 전기차는,"['볼보', '폴스타', '안전', '자동차', '위', '국산', '전기차']",볼보 폴스타 안전 자동차 위 국산 전기차
1,2023-01,현대제철 친환경 전기차 시장 공략 수익성 회복 만전,"['현대제철', '친환경', '전기차', '시장', '공략', '수익', '회복',...",현대제철 친환경 전기차 시장 공략 수익 회복 전
2,2023-01,테슬라 내리자 포드도 세일 전기차 치킨게임 시작,"['테슬라', '내리', '포드', '세일', '전기', '차', '치킨', '게임...",테슬라 내리 포드 세일 전기 차 치킨 게임 시작
3,2023-01,SK온 현대차 아이오닉 배터리 서산에서 만든다,"['SK', '현대차', '아이오닉', '배터리', '서산']",SK 현대차 아이오닉 배터리 서산
4,2023-01,공격적 수주 현대모비스 해외 일감 배 껑충 사상 최대,"['공격', '수주', '현대모비스', '해외', '일감', '배', '껑충', '...",공격 수주 현대모비스 해외 일감 배 껑충 사상 최대
...,...,...,...,...
2295,2024-11,현대차 말레이시아에 억 투자 스타리아 위탁생산,"['현대차', '말레이시아', '투자', '스타리아', '위탁', '생산']",현대차 말레이시아 투자 스타리아 위탁 생산
2296,2024-11,현대차 말레이시아에 천억 투자해 스타리아 위탁생산,"['현대차', '말레이시아', '투자', '스타리아', '위탁', '생산']",현대차 말레이시아 투자 스타리아 위탁 생산
2297,2024-11,현대차 말레이시아에 억원 투자 만대 생산체제 구축,"['현대차', '말레이시아', '투자', '생산', '체제', '구축']",현대차 말레이시아 투자 생산 체제 구축
2298,2024-11,현대모비스 내년 CES 참가 글로벌 자동차 고객 확보 나선다,"['현대모비스', '내년', 'CES', '참가', '글로벌', '자동차', '고객...",현대모비스 내년 CES 참가 글로벌 자동차 고객 확보


In [ ]:
news = pd.read_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/naver_news.csv")

In [ ]:
news

,date,title,source,contents,link,month
0,2023-01-31,볼보·폴스타 제쳤다…안전한 자동차 1위 오른 국산 전기차는,중앙일보언론사 선정,평가대상 전기차는 ▶현대 아이오닉6 ▶기아 니로EV ▶볼보 XC40 리차지 ▶폴스타...,https://www.joongang.co.kr/article/25137117,2023-01
1,2023-01-31,"현대제철, 친환경·전기차 시장 공략...""수익성 회복 만전""",머니투데이,"체코 핫스탬핑 공장 증설을 통해 유럽향 핫스탬핑 수주 및 판매를 강화하고, 미국 현...",http://news.mt.co.kr/mtview.php?no=20230131152...,2023-01
2,2023-01-31,테슬라 내리자 포드도 '세일'…전기차 치킨게임 시작,매일경제,미국 전기차 시장에서 현대자동차와 기아의 주력 모델인 아이오닉5와 EV6는 상대적으...,https://www.mk.co.kr/article/10625406,2023-01
3,2023-01-31,"SK온, 현대차 아이오닉7 배터리 서산에서 만든다",조선비즈언론사 선정,업계 관계자는 “현대차 전기차 생산능력 확대와 더불어 최근 협력 관계가 깊어지고 있...,https://biz.chosun.com/industry/car/2023/01/31...,2023-01
4,2023-01-31,"'공격적 수주' 현대모비스, 해외 일감 2배 껑충...'사상 최대'",파이낸셜뉴스언론사 선정,현대모비스는 약 70% 정도인 현대차·기아 의존도를 낮추기 위해 최근 수년간 공격적...,http://www.fnnews.com/news/202301311448053294,2023-01
...,...,...,...,...,...,...
2295,2024-11-28,"현대차, 말레이시아에 6700억 투자…스타리아 위탁생산",현대경제신문,"현대차는 말레이시아 내 전기차 판매 확대, 충전 인프라 건설, 배터리 생산시설 구축...",http://www.finomy.com/news/articleView.html?id...,2024-11
2296,2024-11-28,"현대차, 말레이시아에 6천억 투자해 스타리아 위탁생산",딜라이트닷넷,"현대차는 말레이시아 내 전기차 판매 확대, 충전 인프라 건설, 배터리 생산시설 구축...",https://www.delighti.co.kr/news/articleView.ht...,2024-11
2297,2024-11-28,"현대차, 말레이시아에 6800억원 투자…年 2만대 생산체제 구축",전자신문,전기차 수요 확대에 대응해 충전 인프라와 배터리 생산시설 등 전기차 생태계 조성도 ...,https://www.etnews.com/20241126000134,2024-11
2298,2024-11-28,"현대모비스, 내년 CES 참가…글로벌 車 고객 확보 나선다",이데일리,(사진=현대모비스) 27일 업계에 따르면 현대모비스는 ‘CES 2025’에 부스를 ...,http://www.edaily.co.kr/news/newspath.asp?news...,2024-11


In [ ]:
news = news[['month', 'title']]

In [ ]:
news

,month,title
0,2023-01,볼보·폴스타 제쳤다…안전한 자동차 1위 오른 국산 전기차는
1,2023-01,"현대제철, 친환경·전기차 시장 공략...""수익성 회복 만전"""
2,2023-01,테슬라 내리자 포드도 '세일'…전기차 치킨게임 시작
3,2023-01,"SK온, 현대차 아이오닉7 배터리 서산에서 만든다"
4,2023-01,"'공격적 수주' 현대모비스, 해외 일감 2배 껑충...'사상 최대'"
...,...,...
2295,2024-11,"현대차, 말레이시아에 6700억 투자…스타리아 위탁생산"
2296,2024-11,"현대차, 말레이시아에 6천억 투자해 스타리아 위탁생산"
2297,2024-11,"현대차, 말레이시아에 6800억원 투자…年 2만대 생산체제 구축"
2298,2024-11,"현대모비스, 내년 CES 참가…글로벌 車 고객 확보 나선다"


In [ ]:
!pip install hanja

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 16.3 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=fff3cb1b6fb38395141768db93aad5f61a653f76d3f8d851699a84b52d94dd26
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2


In [ ]:
import re
import hanja

In [ ]:
def change_hanja_etc(x):
    x = re.sub('↑', '증가', x)
    x = re.sub('↓', '감소', x)
    x = re.sub('車', '자동차', x)
    x = re.sub('美', '미국', x)
    x = re.sub('韓', '한국', x)
    x = re.sub('日', '일본', x)
    x = re.sub('中', '중국', x)
    x = re.sub('英', '영국', x)
    x = re.sub('獨', '독일', x)
    x = re.sub("伊", "이탈리아", x)
    x = re.sub("佛", "프랑스", x)
    x = re.sub("亞", "아시아", x)
    x = re.sub("印", "인도", x)
    x = re.sub("比", "북한", x)
    x = re.sub("新", "새로운 ", x)
    x = re.sub('年', '매년', x)
    x = re.sub("前", '이전', x)
    x = re.sub("反", "반대", x)
    x = re.sub("强", "강자", x)
    x = re.sub("道", "도로", x)
    x = re.sub("業", "업적", x)
    x = re.sub("賞", "상", x)
    x = re.sub("弗", "달러", x)
    x = re.sub("對", "대결", x)
    x = re.sub('株', '주식 ', x)
    x = re.sub('州', '주', x)  # 외국 지역 단위 주
    x = re.sub('市', '시', x)
    x = re.sub('現', '현재', x)
    x = re.sub('社', '회사', x)
    x = hanja.translate(x, 'substitution')
    return x

In [ ]:
def clean_text(title):
    # 한글, 영어, 숫자, 공백만 남기고 나머지는 제거
    title_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9\\s]", " ", title)  # 숫자 포함
    title_clean = re.sub(" +", " ", title_clean)  # 중복 공백 제거
    title_clean = title_clean.strip()  # 문자열 양쪽 공백 제거
    return title_clean

In [ ]:
def preprocessing_news(title):
    title = change_hanja_etc(title)
    title = clean_text(title)
    return title

In [ ]:
news['title'] = news['title'].apply(preprocessing_news)

In [ ]:
news

,month,title
0,2023-01,볼보 폴스타 제쳤다 안전한 자동차 1위 오른 국산 전기차는
1,2023-01,현대제철 친환경 전기차 시장 공략 수익성 회복 만전
2,2023-01,테슬라 내리자 포드도 세일 전기차 치킨게임 시작
3,2023-01,SK온 현대차 아이오닉7 배터리 서산에서 만든다
4,2023-01,공격적 수주 현대모비스 해외 일감 2배 껑충 사상 최대
...,...,...
2295,2024-11,현대차 말레이시아에 6700억 투자 스타리아 위탁생산
2296,2024-11,현대차 말레이시아에 6천억 투자해 스타리아 위탁생산
2297,2024-11,현대차 말레이시아에 6800억원 투자 2만대 생산체제 구축
2298,2024-11,현대모비스 내년 CES 참가 글로벌 자동차 고객 확보 나선다


## way1) 한글 감성어 사전  
군산대학교 감성사전 이용

In [ ]:
import json

In [ ]:
with open('/content/gdrive/MyDrive/통계분석실습/기말프로젝트/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
  SentiWord_info = json.load(f)

In [ ]:
sentiword_dic = pd.DataFrame(SentiWord_info)

In [ ]:
sentiword_dic

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14849,반신반의하다,반신반의,0
14850,신비롭다,신비,1
14851,아리송하다,아리송,-1
14852,알쏭하다,알쏭하,-1


In [ ]:
sentiword_dic['polarity'].unique()

array(['1', '-1', '0', '-2', '2'], dtype=object)



---



### 단어 추가

In [ ]:
sentiword_dic[sentiword_dic['word'] == "최고"]

,word,word_root,polarity
12687,최고,최고,2


In [ ]:
new_word = pd.DataFrame({
    "word": ['버틴다', '돌파', '석권', '연전연패', '위', '수익', '판매', '저조', '수상', '껑충', '트럼프', '꽃길', '성장', '강화', '선점', '질주', '달성', '극복', '증가', '극찬', '부과', '극대', '최저', '부진', '최악', '극찬', '칭찬', '최대', '캐즘', '회복', '확보', '투자', '리콜'],
    "word_root": ['버틴다', '돌파', '석권', '연전연패', '위', '수익', '판매', '저조', '수상', '껑충', '트럼프', '꽃길', '성장', '강화', '선점', '질주', '달성', '극복', '증가', '극찬', '부과', '극대', '최저', '부진', '최악', '극찬', '칭찬', '최대', '캐즘', '회복', '확보', '투자', '리콜'],
    "polarity": [-1, 2, 2, -2, 1, 2, 1, -2, 2, 2, -2, 2, 2, 2, 2, 2, 2, 1, 2, 2, -1, 2, -2, -1, -2, 2, 1, 2, -2, 1, 1, 1, -2]
})

In [ ]:
new_word = pd.DataFrame({
    "word": ['캐즘', '최대', '회복', '확보', '투자', '리콜', '버틴다', '돌파', '석권', '연전연패', '위', '수익', '판매', '저조', '수상', '껑충', '트럼프', '꽃길', '성장', '강화', '선점', '질주', '달성', '극복', '증가', '극찬', '부과', '극대', '최저', '부진', '최악', '극찬', '칭찬'],
    "word_root": ['캐즘', '최대', '회복', '확보', '투자', '리콜', '버틴다', '돌파', '석권', '연전연패', '위', '수익', '판매', '저조', '수상', '껑충', '트럼프', '꽃길', '성장', '강화', '선점', '질주', '달성', '극복', '증가', '극찬', '부과', '극대', '최저', '부진', '최악', '극찬', '칭찬'],
    "polarity": [-2, 2, 1, 1, 1, -2, -1, 2, 2, -2, 1, 2, 1, -2, 2, 2, -2, 2, 2, 2, 2, 2, 2, 1, 2, 2, -1, 2, -2, -1, -2, 2, 1]
})

In [ ]:
sentiword_dic = pd.concat([sentiword_dic, new_word], ignore_index=True)

In [ ]:
sentiword_dic

,word,word_root,polarity
0,(-;,(,1
1,(;_;),(;_;),-1
2,(^^),(^^),1
3,(^-^),(^-^),1
4,(^^*,(,1
...,...,...,...
14882,캐즘,캐즘,-2
14883,회복,회복,1
14884,확보,확보,1
14885,투자,투자,1


In [ ]:
sentiword_dic.tail(20)

,word,word_root,polarity
14867,저조,저조,-2
14868,수상,수상,2
14869,껑충,껑충,2
14870,트럼프,트럼프,-2
14871,꽃길,꽃길,2
14872,성장,성장,2
14873,강화,강화,2
14874,선점,선점,2
14875,질주,질주,2
14876,달성,달성,2




---



In [ ]:
sentiword_dic['polarity'] = pd.to_numeric(sentiword_dic['polarity'])

In [ ]:
sentiword_dic.dtypes

,0
word,object
word_root,object
polarity,int64


In [ ]:
sentiment_dict = dict(zip(sentiword_dic['word'], sentiword_dic['polarity']))

In [ ]:
def calculate_sentiment(tokens):
  sentiment = 0
  for word in tokens:
    sentiment += sentiment_dict.get(word, 0)
  return sentiment

In [ ]:
naver_news['sentiment_score'] = naver_news['token_lst'].apply(calculate_sentiment)

In [ ]:
def classify_sentiment(score):
  if score > 0:
    return 1
  elif score < 0:
    return -1
  else:
    return 0

In [ ]:
naver_news['sentiment_label'] = naver_news['sentiment_score'].apply(classify_sentiment)

In [ ]:
naver_news

,month,title,token_lst,token,sentiment_score,sentiment_label
0,2023-01,볼보 폴스타 제쳤다 안전한 자동차 위 오른 국산 전기차는,"['볼보', '폴스타', '안전', '자동차', '위', '국산', '전기차']",볼보 폴스타 안전 자동차 위 국산 전기차,1,1
1,2023-01,현대제철 친환경 전기차 시장 공략 수익성 회복 만전,"['현대제철', '친환경', '전기차', '시장', '공략', '수익', '회복',...",현대제철 친환경 전기차 시장 공략 수익 회복 전,0,0
2,2023-01,테슬라 내리자 포드도 세일 전기차 치킨게임 시작,"['테슬라', '내리', '포드', '세일', '전기', '차', '치킨', '게임...",테슬라 내리 포드 세일 전기 차 치킨 게임 시작,0,0
3,2023-01,SK온 현대차 아이오닉 배터리 서산에서 만든다,"['SK', '현대차', '아이오닉', '배터리', '서산']",SK 현대차 아이오닉 배터리 서산,0,0
4,2023-01,공격적 수주 현대모비스 해외 일감 배 껑충 사상 최대,"['공격', '수주', '현대모비스', '해외', '일감', '배', '껑충', '...",공격 수주 현대모비스 해외 일감 배 껑충 사상 최대,-2,-1
...,...,...,...,...,...,...
2295,2024-11,현대차 말레이시아에 억 투자 스타리아 위탁생산,"['현대차', '말레이시아', '투자', '스타리아', '위탁', '생산']",현대차 말레이시아 투자 스타리아 위탁 생산,1,1
2296,2024-11,현대차 말레이시아에 천억 투자해 스타리아 위탁생산,"['현대차', '말레이시아', '투자', '스타리아', '위탁', '생산']",현대차 말레이시아 투자 스타리아 위탁 생산,1,1
2297,2024-11,현대차 말레이시아에 억원 투자 만대 생산체제 구축,"['현대차', '말레이시아', '투자', '생산', '체제', '구축']",현대차 말레이시아 투자 생산 체제 구축,0,0
2298,2024-11,현대모비스 내년 CES 참가 글로벌 자동차 고객 확보 나선다,"['현대모비스', '내년', 'CES', '참가', '글로벌', '자동차', '고객...",현대모비스 내년 CES 참가 글로벌 자동차 고객 확보,-2,-1


=> 너무 성능이 안좋음

## way2) KOBERT사전학습 모델 사용    
  
참고:   
- https://bbarry-lee.github.io/ai-tech/KoBERT%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%9C-%EA%B0%90%EC%A0%95%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EA%B5%AC%ED%98%84.html  
- https://velog.io/@amg1682/KoBERT-%EC%98%A4%EB%A5%98-%ED%95%B4%EA%B2%B0

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-vy__06gi/kobert-tokenizer_068b72c0733d4a64b92d0676c47a0111
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-vy__06gi/kobert-tokenizer_068b72c0733d4a64b92d0676c47a0111
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done


In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

numpy 버전과 gluconnlp 버전 호환성 이슈

In [ ]:
print(np.__version__)

1.26.4


In [ ]:
!pip install numpy==1.19.5

  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (numpy)


In [ ]:
!pip install gluonnlp==0.10.0

In [ ]:
# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


AttributeError: 'spacy.vocab.Vocab' object has no attribute 'BERTVocab'

In [ ]:
# koBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

  Cloning https://github.com/dmlc/gluon-nlp.git to /tmp/pip-req-build-unxjedpl
  Running command git clone --filter=blob:none --quiet https://github.com/dmlc/gluon-nlp.git /tmp/pip-req-build-unxjedpl
  Resolved https://github.com/dmlc/gluon-nlp.git to commit 14553a0c19790a3b03e8107471c987c7aa4f8faa
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 44.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.8/883.8 kB 41.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.7/508.7 kB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.2/184.2 kB 12.6 MB/s eta 0

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
model, vocab = get_pytorch_kobert_model()
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
def sentiment_analysis(model, tokenizer, texts, max_len=64):
    labels = [0] * len(texts)  # 임시 라벨, 추후 예측을 위해 사용할 것
    dataset = SentimentDataset(texts, labels, tokenizer, max_len)
    dataloader = DataLoader(dataset, batch_size=32)

    model.eval()  # 모델을 평가 모드로 전환

    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
            attention_mask = batch['attention_mask'].to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

            output = model(input_ids, attention_mask=attention_mask)
            logits = output[0]
            predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())  # 예측값을 리스트에 추가

    return predictions

## way3) chatGPT api로 영어 번역시킨 후 감정분석 라이브러리 사용

#### 1) 네이버 뉴스 -> 영어 번역

In [ ]:
news

,month,title
0,2023-01,볼보 폴스타 제쳤다 안전한 자동차 1위 오른 국산 전기차는
1,2023-01,현대제철 친환경 전기차 시장 공략 수익성 회복 만전
2,2023-01,테슬라 내리자 포드도 세일 전기차 치킨게임 시작
3,2023-01,SK온 현대차 아이오닉7 배터리 서산에서 만든다
4,2023-01,공격적 수주 현대모비스 해외 일감 2배 껑충 사상 최대
...,...,...
2295,2024-11,현대차 말레이시아에 6700억 투자 스타리아 위탁생산
2296,2024-11,현대차 말레이시아에 6천억 투자해 스타리아 위탁생산
2297,2024-11,현대차 말레이시아에 6800억원 투자 2만대 생산체제 구축
2298,2024-11,현대모비스 내년 CES 참가 글로벌 자동차 고객 확보 나선다


In [ ]:
from openai import OpenAI

In [ ]:
# OpenAI API Key
client = OpenAI(api_key = "####")

In [ ]:
def gpt_translate_to_english(text):
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a translator."},
                {
                    "role": "user",
                    "content": f"Translate the following text to English. {text}"
                }
            ]
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
translated_text = gpt_translate_to_english("안녕하세요, 번역 테스트를 하고 있습니다.")
print(translated_text)

Hello, I am conducting a translation test.


In [ ]:
news['title_en'] = news['title'].apply(gpt_translate_to_english)

In [ ]:
news

,month,title,title_en
0,2023-01,볼보 폴스타 제쳤다 안전한 자동차 1위 오른 국산 전기차는,The domestic electric car that surpassed Volvo...
1,2023-01,현대제철 친환경 전기차 시장 공략 수익성 회복 만전,Hyundai Steel is striving to restore profitabi...
2,2023-01,테슬라 내리자 포드도 세일 전기차 치킨게임 시작,"Tesla drops prices, Ford follows with discount..."
3,2023-01,SK온 현대차 아이오닉7 배터리 서산에서 만든다,SK On's Hyundai Ioniq 7 Battery to be Manufact...
4,2023-01,공격적 수주 현대모비스 해외 일감 2배 껑충 사상 최대,Aggressive Orders: Hyundai Mobis Doubles Overs...
...,...,...,...
2295,2024-11,현대차 말레이시아에 6700억 투자 스타리아 위탁생산,Hyundai invests 670 billion won in Malaysia fo...
2296,2024-11,현대차 말레이시아에 6천억 투자해 스타리아 위탁생산,Hyundai Motor Company to invest 600 billion wo...
2297,2024-11,현대차 말레이시아에 6800억원 투자 2만대 생산체제 구축,Hyundai invests 680 billion KRW in Malaysia to...
2298,2024-11,현대모비스 내년 CES 참가 글로벌 자동차 고객 확보 나선다,Hyundai Mobis to Participate in CES Next Year ...


In [ ]:
news.to_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/naver_news_translate.csv", index=False)

#### 2) 해외 뉴스 -> 요약

In [ ]:
df1 = pd.read_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/ev_market.csv")
df2 = pd.read_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/insiedEV_hyd.csv")
df3 = pd.read_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/insiedEV_kia.csv")

In [ ]:
df = pd.concat([df1, df2, df3], ignore_index=True)
df

,date,title,article
0,20230206,Global EV Sales for 2022,Global EV sales continued strong. A total of 1...
1,20230412,Another Record Broken: Q1 Electric Vehicle Sal...,"For more than a year now, the Industry Insight..."
2,20230612,Electric Vehicle Sales in Q2 Strike Another Re...,"Nearly 300,000 new electric vehicles (EVs) – f..."
3,20230817,Global EV Sales for 2023 H1,Global EV sales continue strong. A total of 6 ...
4,20231012,"Another Quarter, Another Record: EV Sales in t...",Electric vehicle (EV) sales volumes set anothe...
...,...,...,...
78,20230620,Expanded Kia EV Lineup To Include Everything F...,"Kia, the Korean car brand that recently reveal..."
79,20230620,2024 Kia EV9 Deliveries Have Already Begun In ...,Kia announced that deliveries of the EV9 three...
80,20230517,UPDATE: Kia Makes 2023 EV6 More Attractive To ...,UPDATE: A Kia America spokesperson told Inside...
81,20230510,See How The Kia EV9 Was Tested To Ensure It Ca...,The KIA EV9 will become the company’s first-ev...


In [ ]:
df.dtypes

,0
date,int64
title,object
article,object


In [ ]:
df = df.sort_values(by=['date']).reset_index(drop=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'], format="%Y%m%d")
df['date'] = df['date'].dt.strftime("%Y-%m")

In [ ]:
df

,date,title,article
0,2023-02,Global EV Sales for 2022,Global EV sales continued strong. A total of 1...
1,2023-02,Hyundai USA Launches Evolve+ EV Subscription P...,Hyundai has introduced an EV subscription serv...
2,2023-03,Hyundai Kona N Electric May Be On The Way,"Hyundai’s rollout of the new, second-generatio..."
3,2023-03,Hyundai Surprised At The Success Of The Ioniq ...,The Hyundai Ioniq 5 is performing better than ...
4,2023-03,Hyundai Finds Causes For Ioniq 5's 12V Battery...,Some Hyundai Ioniq 5 owners are complaining th...
...,...,...,...
78,2024-11,Hyundai Ioniq 9: A Three-Row EV With 335 Miles...,"Meet the Hyundai Ioniq 9, the brand's first th..."
79,2024-11,'It Would Just Be Dumb': Kia Exec On Plan To K...,Kia America's COO said that eliminating the ta...
80,2024-11,GM And Hyundai EV Pickup Could Be In Development,General Motors and Hyundai are reportedly team...
81,2024-11,Kia Eyes PHEVs With 60 Miles Of Electric Range...,Kia's plug-in hybrids might offer 60 miles of ...


In [ ]:
def gpt_summarize_article(text):
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an assistant specializing in summarization."},
                {
                    "role": "user",
                    "content": f"Summarize this article into one sentence, focusing on what it says about the EV market sales and Hyundai-Kia EVs sales. Keep it concise: {text}"}
            ],
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
df['summary'] = df['article'].apply(gpt_summarize_article)

In [ ]:
df

,date,title,article,summary
0,2023-02,Global EV Sales for 2022,Global EV sales continued strong. A total of 1...,Global EV sales grew by 55% in 2022 with 10.5 ...
1,2023-02,Hyundai USA Launches Evolve+ EV Subscription P...,Hyundai has introduced an EV subscription serv...,The article highlights Hyundai's introduction ...
2,2023-03,Hyundai Kona N Electric May Be On The Way,"Hyundai’s rollout of the new, second-generatio...",Hyundai is evaluating market reaction to the I...
3,2023-03,Hyundai Surprised At The Success Of The Ioniq ...,The Hyundai Ioniq 5 is performing better than ...,The Hyundai Ioniq 5 has exceeded sales expecta...
4,2023-03,Hyundai Finds Causes For Ioniq 5's 12V Battery...,Some Hyundai Ioniq 5 owners are complaining th...,"The article reports that Hyundai-Kia EVs, spec..."
...,...,...,...,...
78,2024-11,Hyundai Ioniq 9: A Three-Row EV With 335 Miles...,"Meet the Hyundai Ioniq 9, the brand's first th...","The Hyundai Ioniq 9, a new three-row electric ..."
79,2024-11,'It Would Just Be Dumb': Kia Exec On Plan To K...,Kia America's COO said that eliminating the ta...,The potential elimination of the EV tax credit...
80,2024-11,GM And Hyundai EV Pickup Could Be In Development,General Motors and Hyundai are reportedly team...,The article reports that while Hyundai and GM ...
81,2024-11,Kia Eyes PHEVs With 60 Miles Of Electric Range...,Kia's plug-in hybrids might offer 60 miles of ...,The article highlights that Kia plans to nearl...


In [ ]:
df.to_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/journal_summarize.csv", index=False)

#### 감정분석 라이브러리 사용

In [ ]:
news = pd.read_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/naver_news_translate.csv")
df = pd.read_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/journal_summarize.csv")

In [ ]:
!pip install textblob
!pip install vaderSentiment

In [ ]:
!pip install flair

In [ ]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from flair.models import TextClassifier
from flair.data import Sentence

In [ ]:
def sentiment_analysis_textblob(text):
  blob = TextBlob(text)
  polarity = blob.sentiment.polarity

  if polarity > 0:
    return 1
  elif polarity < 0:
    return -1
  else:
    return 0

In [ ]:
analyzer = SentimentIntensityAnalyzer()

def sentiment_analysis_vader(text):
  sentiment = analyzer.polarity_scores(text)
  if sentiment['compound'] > 0.05:
    return 1
  elif sentiment['compound'] < -0.05:
    return -1
  else:
    return 0

In [ ]:
classifier = TextClassifier.load('sentiment')

def sentiment_analysis_flair(text):
  sentence = Sentence(text)
  classifier.predict(sentence)
  sentiment = sentence.labels[0].value
  if sentiment == "POSITIVE":
    return 1
  elif sentiment == "NEGATIVE":
    return -1
  else:
    return 0

In [ ]:
# 1. naver_news_translate
news['sentiment_textblob'] = news['title_en'].apply(sentiment_analysis_textblob)
news['sentiment_vader'] = news['title_en'].apply(sentiment_analysis_vader)
news['sentiment_knu'] = naver_news['sentiment_label']
news['sentiment_flair'] = news['title_en'].apply(sentiment_analysis_flair)

In [ ]:
news

,month,title,title_en,sentiment_textblob,sentiment_vader,sentiment_knu,sentiment_flair
0,2023-01,볼보 폴스타 제쳤다 안전한 자동차 1위 오른 국산 전기차는,The domestic electric car that surpassed Volvo...,0,1,1,1
1,2023-01,현대제철 친환경 전기차 시장 공략 수익성 회복 만전,Hyundai Steel is striving to restore profitabi...,0,1,0,-1
2,2023-01,테슬라 내리자 포드도 세일 전기차 치킨게임 시작,"Tesla drops prices, Ford follows with discount...",0,-1,0,-1
3,2023-01,SK온 현대차 아이오닉7 배터리 서산에서 만든다,SK On's Hyundai Ioniq 7 Battery to be Manufact...,0,0,0,1
4,2023-01,공격적 수주 현대모비스 해외 일감 2배 껑충 사상 최대,Aggressive Orders: Hyundai Mobis Doubles Overs...,1,-1,-1,1
...,...,...,...,...,...,...,...
2295,2024-11,현대차 말레이시아에 6700억 투자 스타리아 위탁생산,Hyundai invests 670 billion won in Malaysia fo...,0,1,1,1
2296,2024-11,현대차 말레이시아에 6천억 투자해 스타리아 위탁생산,Hyundai Motor Company to invest 600 billion wo...,0,1,1,1
2297,2024-11,현대차 말레이시아에 6800억원 투자 2만대 생산체제 구축,Hyundai invests 680 billion KRW in Malaysia to...,0,0,0,1
2298,2024-11,현대모비스 내년 CES 참가 글로벌 자동차 고객 확보 나선다,Hyundai Mobis to Participate in CES Next Year ...,1,1,-1,1


In [ ]:
# 과반수로 최종 결과 정하기
def majority_vote(row):
  counts = row.value_counts()
  max_count = counts.max()
  max_sentiments = counts[counts == max_count].index.tolist()

  # 동률
  if len(max_sentiments) > 1:
    return 0
  else:
    return max_sentiments[0]

In [ ]:
sentiment_columns = ['sentiment_textblob', 'sentiment_vader', 'sentiment_knu', 'sentiment_flair']

news['majority_sentiment'] = news[sentiment_columns].apply(majority_vote, axis=1)

In [ ]:
news

,month,title,title_en,sentiment_textblob,sentiment_vader,sentiment_knu,sentiment_flair,majority_sentiment
0,2023-01,볼보 폴스타 제쳤다 안전한 자동차 1위 오른 국산 전기차는,The domestic electric car that surpassed Volvo...,0,1,1,1,1
1,2023-01,현대제철 친환경 전기차 시장 공략 수익성 회복 만전,Hyundai Steel is striving to restore profitabi...,0,1,0,-1,0
2,2023-01,테슬라 내리자 포드도 세일 전기차 치킨게임 시작,"Tesla drops prices, Ford follows with discount...",0,-1,0,-1,0
3,2023-01,SK온 현대차 아이오닉7 배터리 서산에서 만든다,SK On's Hyundai Ioniq 7 Battery to be Manufact...,0,0,0,1,0
4,2023-01,공격적 수주 현대모비스 해외 일감 2배 껑충 사상 최대,Aggressive Orders: Hyundai Mobis Doubles Overs...,1,-1,-1,1,0
...,...,...,...,...,...,...,...,...
2295,2024-11,현대차 말레이시아에 6700억 투자 스타리아 위탁생산,Hyundai invests 670 billion won in Malaysia fo...,0,1,1,1,1
2296,2024-11,현대차 말레이시아에 6천억 투자해 스타리아 위탁생산,Hyundai Motor Company to invest 600 billion wo...,0,1,1,1,1
2297,2024-11,현대차 말레이시아에 6800억원 투자 2만대 생산체제 구축,Hyundai invests 680 billion KRW in Malaysia to...,0,0,0,1,0
2298,2024-11,현대모비스 내년 CES 참가 글로벌 자동차 고객 확보 나선다,Hyundai Mobis to Participate in CES Next Year ...,1,1,-1,1,1


In [ ]:
def calculate_sentiment_ratio(group):
  total = len(group)
  p = (group == 1).sum() / total if total > 0 else 0
  n = (group == -1).sum() / total if total > 0 else 0
  pnr = (group == 1).sum() / ((group == -1).sum() + 0.1)

  return pd.Series({
      "positive_ratio": round(p, 2),
      "negative_ratio": round(n, 2),
      "pnr": round(pnr, 2)
  })

In [ ]:
result_naver = news.groupby("month")['majority_sentiment'].apply(calculate_sentiment_ratio).unstack().reset_index()

In [ ]:
result_naver

,month,positive_ratio,negative_ratio,pnr
0,2023-01,0.33,0.19,1.73
1,2023-02,0.32,0.09,3.52
2,2023-03,0.36,0.10,3.56
3,2023-04,0.39,0.05,7.65
4,2023-05,0.25,0.11,2.25
5,2023-06,0.28,0.04,6.83
6,2023-07,0.36,0.12,2.98
7,2023-08,0.36,0.11,3.24
8,2023-09,0.33,0.12,2.73
9,2023-10,0.30,0.11,2.70


In [ ]:
# 2. journal_summarize
df['sentiment_textblob'] = df['summary'].apply(sentiment_analysis_textblob)
df['sentiment_vader'] = df['summary'].apply(sentiment_analysis_vader)
df['sentiment_flair'] = df['summary'].apply(sentiment_analysis_flair)

In [ ]:
df

,date,title,article,summary,sentiment_textblob,sentiment_vader,sentiment_flair
0,2023-02,Global EV Sales for 2022,Global EV sales continued strong. A total of 1...,Global EV sales grew by 55% in 2022 with 10.5 ...,0,0,1
1,2023-02,Hyundai USA Launches Evolve+ EV Subscription P...,Hyundai has introduced an EV subscription serv...,The article highlights Hyundai's introduction ...,1,1,1
2,2023-03,Hyundai Kona N Electric May Be On The Way,"Hyundai’s rollout of the new, second-generatio...",Hyundai is evaluating market reaction to the I...,0,1,1
3,2023-03,Hyundai Surprised At The Success Of The Ioniq ...,The Hyundai Ioniq 5 is performing better than ...,The Hyundai Ioniq 5 has exceeded sales expecta...,1,1,1
4,2023-03,Hyundai Finds Causes For Ioniq 5's 12V Battery...,Some Hyundai Ioniq 5 owners are complaining th...,"The article reports that Hyundai-Kia EVs, spec...",1,-1,-1
...,...,...,...,...,...,...,...
78,2024-11,Hyundai Ioniq 9: A Three-Row EV With 335 Miles...,"Meet the Hyundai Ioniq 9, the brand's first th...","The Hyundai Ioniq 9, a new three-row electric ...",1,1,1
79,2024-11,'It Would Just Be Dumb': Kia Exec On Plan To K...,Kia America's COO said that eliminating the ta...,The potential elimination of the EV tax credit...,1,1,-1
80,2024-11,GM And Hyundai EV Pickup Could Be In Development,General Motors and Hyundai are reportedly team...,The article reports that while Hyundai and GM ...,-1,-1,-1
81,2024-11,Kia Eyes PHEVs With 60 Miles Of Electric Range...,Kia's plug-in hybrids might offer 60 miles of ...,The article highlights that Kia plans to nearl...,1,-1,1


In [ ]:
sentiment_columns = ["sentiment_textblob", "sentiment_vader", "sentiment_flair"]

df['majority_sentiment'] = df[sentiment_columns].apply(majority_vote, axis=1)

In [ ]:
df

,date,title,article,summary,sentiment_textblob,sentiment_vader,sentiment_flair,majority_sentiment
0,2023-02,Global EV Sales for 2022,Global EV sales continued strong. A total of 1...,Global EV sales grew by 55% in 2022 with 10.5 ...,0,0,1,0
1,2023-02,Hyundai USA Launches Evolve+ EV Subscription P...,Hyundai has introduced an EV subscription serv...,The article highlights Hyundai's introduction ...,1,1,1,1
2,2023-03,Hyundai Kona N Electric May Be On The Way,"Hyundai’s rollout of the new, second-generatio...",Hyundai is evaluating market reaction to the I...,0,1,1,1
3,2023-03,Hyundai Surprised At The Success Of The Ioniq ...,The Hyundai Ioniq 5 is performing better than ...,The Hyundai Ioniq 5 has exceeded sales expecta...,1,1,1,1
4,2023-03,Hyundai Finds Causes For Ioniq 5's 12V Battery...,Some Hyundai Ioniq 5 owners are complaining th...,"The article reports that Hyundai-Kia EVs, spec...",1,-1,-1,-1
...,...,...,...,...,...,...,...,...
78,2024-11,Hyundai Ioniq 9: A Three-Row EV With 335 Miles...,"Meet the Hyundai Ioniq 9, the brand's first th...","The Hyundai Ioniq 9, a new three-row electric ...",1,1,1,1
79,2024-11,'It Would Just Be Dumb': Kia Exec On Plan To K...,Kia America's COO said that eliminating the ta...,The potential elimination of the EV tax credit...,1,1,-1,1
80,2024-11,GM And Hyundai EV Pickup Could Be In Development,General Motors and Hyundai are reportedly team...,The article reports that while Hyundai and GM ...,-1,-1,-1,-1
81,2024-11,Kia Eyes PHEVs With 60 Miles Of Electric Range...,Kia's plug-in hybrids might offer 60 miles of ...,The article highlights that Kia plans to nearl...,1,-1,1,1


In [ ]:
result_foreign = df.groupby("date")['majority_sentiment'].apply(calculate_sentiment_ratio).unstack().reset_index()

In [ ]:
result_foreign

,date,positive_ratio,negative_ratio,pnr
0,2023-02,0.50,0.00,10.00
1,2023-03,0.75,0.25,2.73
2,2023-04,1.00,0.00,20.00
3,2023-05,0.75,0.00,30.00
4,2023-06,0.88,0.12,6.36
5,2023-08,0.40,0.60,0.65
6,2023-09,1.00,0.00,10.00
7,2023-10,1.00,0.00,20.00
8,2023-11,1.00,0.00,10.00
9,2024-01,0.50,0.50,0.91


## 최종 감성분석 결과 dataframe 만들기

In [ ]:
result_naver.columns = ['month', 'p_naver', 'n_naver', 'pnr_naver']
result_foreign.columns = ['month', 'p_foreign', 'n_foreign', 'pnr_foreign']

In [ ]:
result_naver

,month,p_naver,n_naver,pnr_naver
0,2023-01,0.33,0.19,1.73
1,2023-02,0.32,0.09,3.52
2,2023-03,0.36,0.10,3.56
3,2023-04,0.39,0.05,7.65
4,2023-05,0.25,0.11,2.25
5,2023-06,0.28,0.04,6.83
6,2023-07,0.36,0.12,2.98
7,2023-08,0.36,0.11,3.24
8,2023-09,0.33,0.12,2.73
9,2023-10,0.30,0.11,2.70


In [ ]:
result_foreign

,month,p_foreign,n_foreign,pnr_foreign
0,2023-02,0.50,0.00,10.00
1,2023-03,0.75,0.25,2.73
2,2023-04,1.00,0.00,20.00
3,2023-05,0.75,0.00,30.00
4,2023-06,0.88,0.12,6.36
5,2023-08,0.40,0.60,0.65
6,2023-09,1.00,0.00,10.00
7,2023-10,1.00,0.00,20.00
8,2023-11,1.00,0.00,10.00
9,2024-01,0.50,0.50,0.91


In [ ]:
sentiment_result = pd.merge(result_naver, result_foreign, on="month", how='outer')

In [ ]:
sentiment_result = sentiment_result[:len(sentiment_result) - 1]

In [ ]:
sentiment_result

,month,p_naver,n_naver,pnr_naver,p_foreign,n_foreign,pnr_foreign
0,2023-01,0.33,0.19,1.73,NaN,NaN,NaN
1,2023-02,0.32,0.09,3.52,0.50,0.00,10.00
2,2023-03,0.36,0.10,3.56,0.75,0.25,2.73
3,2023-04,0.39,0.05,7.65,1.00,0.00,20.00
4,2023-05,0.25,0.11,2.25,0.75,0.00,30.00
5,2023-06,0.28,0.04,6.83,0.88,0.12,6.36
6,2023-07,0.36,0.12,2.98,NaN,NaN,NaN
7,2023-08,0.36,0.11,3.24,0.40,0.60,0.65
8,2023-09,0.33,0.12,2.73,1.00,0.00,10.00
9,2023-10,0.30,0.11,2.70,1.00,0.00,20.00


In [ ]:
sentiment_result.to_csv("/content/gdrive/MyDrive/통계분석실습/기말프로젝트/data/sentiment_result.csv", index=False)